In [1]:

import os
from tqdm import tqdm
import cv2
import torch
import numpy as np

In [2]:
gt_path = '../testdata/labels/'
preds_path = '../testdata/preds/'

# get all imgs'name, remove the extension
images_folder1 = [f for f in os.listdir(gt_path) if
                  os.path.isfile(os.path.join(gt_path, f)) and f.lower().endswith(
                      ('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.gif'))]
images_names = [os.path.splitext(f)[0] for f in images_folder1]
tbar = tqdm(images_names)



  0%|          | 0/100 [00:00<?, ?it/s]

# Note
We found that the different data formats made the metrics change, which was not due to an error in our calculations, but was caused by the matching mechanism of the forloop.  
The different data formats are converted to a uniform data format within the code, but there may be some reason (guessing memory discontinuity) that makes `skimage.measure.regionprops` return results in a different order. This causes the order in the distances/iou matrix to change, so the `forloops` give different matching results.  
We will use a small example to demonstrate this phenomenon, or observe it by setting `debug='True'` and then running the data in a different format.

In [3]:
# 1. Definition of the forloops match function.
def forloops_match(dis, thr):
    num_gt, num_pred = dis.shape
    tp = torch.zeros(1)
    fp= torch.zeros(1)
    fn = torch.zeros(1)
    for i in range(num_gt):
        for j in range(num_pred):
            if dis[i,i]< thr:
                tp += 1
                dis[:,j]= torch.inf # mark matched
                break
    fp = num_pred-tp
    fn = num_gt-tp
    pre = torch.divide(tp,tp+fp)
    rec = torch.divide(tp,tp+fn)
    return tp, fp, fn, pre, rec

In [4]:
# Definition of the distances matrix, rows and columns are presented as groundtruth and prediction respectively.
dis =torch.tensor([[0.33, 2],
                  [0.2, 10]])
thr = 3 # threshold for matching

In [5]:
tp, fp, fn, pre, rec = forloops_match(dis,thr)
f"{thr=},{pre=},{rec=},{tp=},{fp=},{fn=}"

'thr=3,pre=tensor([0.5000]),rec=tensor([0.5000]),tp=tensor([1.]),fp=tensor([1.]),fn=tensor([1.])'

In [6]:
# Let's swap the ordering of gt
dis =torch.tensor([[0.2, 10],
                    [0.33, 2],
                  ])
tp, fp, fn, pre, rec = forloops_match(dis,thr)
f"{thr=},{pre=},{rec=},{tp=},{fp=},{fn=}"

'thr=3,pre=tensor([1.]),rec=tensor([1.]),tp=tensor([2.]),fp=tensor([0.]),fn=tensor([0.])'

All the metrics have changed!  
In practice, this may have no effect on the comparison of the algorithms, since in the experiments all the algorithms used `forloops` and the same data format.'  
However, we still want to get fully consistent results, so we will improve this in subsequent releases(development has been completed, but it is still being tested).


# Target Center Metric

In [7]:
from sosmetrics.metrics import TargetPrecisionRecallF1
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []

Metric = TargetPrecisionRecallF1(
    dis_thrs=[1, 10],
    conf_thr=0.5,
    match_alg='forloop',
    # debug = True
    )
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()

Reading image_name=0506: 100%|██████████| 100/100 [00:07<00:00, 13.95it/s]


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
TargetPrecisionRecallF1.update() took 0.00s each time.
+---------+-----+-----+-----+------------------+---------------+----------------+
| Dis-Thr |  TP |  FP |  FN | target_Precision | target_Recall | target_F1score |
+---------+-----+-----+-----+------------------+---------------+----------------+
|   1.0   | 148 | 210 | 170 |     0.41341      |    0.46541    |    0.43787     |
|   2.0   | 236 | 122 |  82 |     0.65922      |    0.74214    |    0.69822     |
|   3.0   | 266 |  92 |  52 |     0.74302      |    0.83648    |    0.78698     |
|   4.0   | 276 |  82 |  42 |     0.77095      |    0.86792    |    0.81657     |
|   5.0   | 282 |  76 |  36 |     0.78771      |    0.88679    |    0.83432     |
|   6.0   | 284 |  74 |  34 |     0.79330      |    0.89308    |    0.84024     |
|   7.0   | 286 |  72 |  32 |     0.79888      |    0.89937    |    0.84615     |
|   8.0 

(array([0.41340782, 0.65921788, 0.74301676, 0.77094972, 0.7877095 ,
        0.79329609, 0.79888268, 0.79888268, 0.79888268, 0.79888268]),
 array([0.46540881, 0.74213836, 0.83647799, 0.86792453, 0.88679245,
        0.89308176, 0.89937107, 0.89937107, 0.89937107, 0.89937107]),
 array([0.43786982, 0.69822485, 0.78698225, 0.81656805, 0.83431953,
        0.84023669, 0.84615385, 0.84615385, 0.84615385, 0.84615385]))

In [8]:
Metric.table

,Dis-Thr,TP,FP,FN,Precision,Recall,F1
0,1.0,74.0,105.0,85.0,0.413408,0.465409,0.437870
1,2.0,118.0,61.0,41.0,0.659218,0.742138,0.698225
2,3.0,133.0,46.0,26.0,0.743017,0.836478,0.786982
3,4.0,138.0,41.0,21.0,0.770950,0.867925,0.816568
4,5.0,141.0,38.0,18.0,0.787709,0.886792,0.834320
5,6.0,142.0,37.0,17.0,0.793296,0.893082,0.840237
6,7.0,143.0,36.0,16.0,0.798883,0.899371,0.846154
7,8.0,143.0,36.0,16.0,0.798883,0.899371,0.846154
8,9.0,143.0,36.0,16.0,0.798883,0.899371,0.846154
9,10.0,143.0,36.0,16.0,0.798883,0.899371,0.846154


## Target Average Precision


In [9]:
from sosmetrics.metrics import TargetAveragePrecision
Metric = TargetAveragePrecision(
    dis_thrs=[1, 10],
    conf_thrs=10,
    match_alg='forloop',
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
TargetAveragePrecision.update() took 0.03s each time.
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr |  1.0   |  2.0   |  3.0   |  4.0   |  5.0   |  6.0   |  7.0   |  8.0   |  9.0   |  10.0  |
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|    AP   | 0.1924 | 0.4892 | 0.6215 | 0.6691 | 0.6985 | 0.6996 | 0.7008 | 0.6921 | 0.6921 | 0.6668 |
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
TargetAveragePrecision(match_alg=forloop)
Test image of list [hwc, hwc, ...]
TargetAveragePrecision.update() took 0.03s each time.
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr |  1.0   |  2.0   |  3.0   |  4.0   |  5.0   |  6.0   |  7.0   |  8.0   |  9

(array([[0.        , 0.41340782, 0.41340782, 0.41340782, 0.41340782,
         0.41340782, 0.41340782, 0.41340782, 0.41340782, 0.41340782],
        [0.        , 0.65921788, 0.65921788, 0.65921788, 0.65921788,
         0.65921788, 0.65921788, 0.65921788, 0.65921788, 0.65921788],
        [0.        , 0.74301676, 0.74301676, 0.74301676, 0.74301676,
         0.74301676, 0.74301676, 0.74301676, 0.74301676, 0.74301676],
        [0.        , 0.77094972, 0.77094972, 0.77094972, 0.77094972,
         0.77094972, 0.77094972, 0.77094972, 0.77094972, 0.77094972],
        [0.        , 0.7877095 , 0.7877095 , 0.7877095 , 0.7877095 ,
         0.7877095 , 0.7877095 , 0.7877095 , 0.7877095 , 0.7877095 ],
        [0.01      , 0.79329609, 0.79329609, 0.79329609, 0.79329609,
         0.79329609, 0.79329609, 0.79329609, 0.79329609, 0.79329609],
        [0.02      , 0.79888268, 0.79888268, 0.79888268, 0.79888268,
         0.79888268, 0.79888268, 0.79888268, 0.79888268, 0.79888268],
        [0.03      , 0.7988

In [10]:
Metric.table

,0,1,2,3,4,5,6,7,8,9
Dis-Thr,1.000000,2.000000,3.000000,4.000000,5.000000,6.00000,7.000000,8.000000,9.000000,10.000000
AP,0.192404,0.489231,0.621517,0.669126,0.698535,0.69961,0.700756,0.692077,0.692077,0.666794


# PD_FA Metric

In [11]:
# For test multiple format of input.
from sosmetrics.metrics import TargetPdPixelFa


gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []

Metric = TargetPdPixelFa(
    dis_thrs=[1, 10],
    conf_thr=0.5,
    match_alg='forloop',
    # debug = True
    )
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png") 
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format
    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
TargetPdPixelFa.update() took 0.01s each time.
+-----------+-----+-----+------+----------+-----------+-------------+
| Threshold |  TD |  AT |  FD  |    NP    | target_Pd |   pixel_Fa  |
+-----------+-----+-----+------+----------+-----------+-------------+
|    1.0    | 148 | 318 | 2540 | 13107200 |  0.46541  | 1.93787e-04 |
|    2.0    | 236 | 318 | 716  | 13107200 |  0.74214  | 5.46265e-05 |
|    3.0    | 266 | 318 | 276  | 13107200 |  0.83648  | 2.10571e-05 |
|    4.0    | 276 | 318 | 278  | 13107200 |  0.86792  | 2.12097e-05 |
|    5.0    | 282 | 318 | 274  | 13107200 |  0.88679  | 2.09045e-05 |
|    6.0    | 284 | 318 | 312  | 13107200 |  0.89308  | 2.38037e-05 |
|    7.0    | 286 | 318 | 310  | 13107200 |  0.89937  | 2.36511e-05 |
|    8.0    | 286 | 318 | 326  | 13107200 |  0.89937  | 2.48718e-05 |
|    9.0    | 286 | 318 | 326  | 13107200 |  0.89937  | 2.48718e-0

(array([0.46540881, 0.74213836, 0.83647799, 0.86792453, 0.88679245,
        0.89308176, 0.89937107, 0.89937107, 0.89937107, 0.89937107]),
 array([1.93786621e-04, 5.46264648e-05, 2.12097168e-05, 2.65502930e-05,
        3.37219238e-05, 4.24194336e-05, 4.24194336e-05, 4.36401367e-05,
        4.36401367e-05, 4.36401367e-05]))

In [12]:
Metric.table

,Dis_thr,TD,AT,FD,NP,target_Pd,pixel_Fa
0,1.0,74.0,159.0,1270.0,6553600.0,0.465409,0.000194
1,2.0,118.0,159.0,358.0,6553600.0,0.742138,0.000055
2,3.0,133.0,159.0,139.0,6553600.0,0.836478,0.000021
3,4.0,138.0,159.0,174.0,6553600.0,0.867925,0.000027
4,5.0,141.0,159.0,221.0,6553600.0,0.886792,0.000034
5,6.0,142.0,159.0,278.0,6553600.0,0.893082,0.000042
6,7.0,143.0,159.0,278.0,6553600.0,0.899371,0.000042
7,8.0,143.0,159.0,286.0,6553600.0,0.899371,0.000044
8,9.0,143.0,159.0,286.0,6553600.0,0.899371,0.000044
9,10.0,143.0,159.0,286.0,6553600.0,0.899371,0.000044


# PixelNormalizeIoU

In [13]:
from sosmetrics.metrics import PixelNormalizedIoU
Metric = PixelNormalizedIoU(
    conf_thr=0.5,
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
PixelNormalizedIoU.update() took 0.00s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.2307  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
Test image of list [hwc, hwc, ...]
PixelNormalizedIoU.update() took 0.00s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.2307  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
Test image_path of list, [img_path, img_path, ...]
PixelNormalizedIoU.update() took 0.00s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.2307  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
Test image of np.array, bhwc
PixelNormalizedIoU.update() took 0.00s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.2307  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
Test image of tensor, bchw
PixelNormalizedIoU.update() took 0.01s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.2307  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
T

0.23073638166447524

In [14]:
Metric.table

,nIoU
0,0.230736


## HybridNormalizedIoU

In [15]:
from sosmetrics.metrics import HybridNormalizedIoU
Metric = HybridNormalizedIoU(
    conf_thr=0.5,
    dis_thrs = [1,10],
    match_alg='forloop',
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    # Metric.update(gt_img, pred_img)
    # Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
# Metric.get()
# Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list, [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()

Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
Test image of list [hwc, hwc, ...]
HybridNormalizedIoU.update() took 0.32s each time.
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr  |  1.0   |  2.0   |  3.0   |  4.0   |  5.0   |  6.0   |  7.0   |  8.0   |  9.0   |  10.0  |
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| nIoU-0.5 | 0.1232 | 0.1902 | 0.2051 | 0.2037 | 0.2030 | 0.2021 | 0.2021 | 0.2017 | 0.2017 | 0.2017 |
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
HybridNormalizedIoU(conf_thr=0.5, match_alg=forloop, second_match=none)
Test image_path of list, [img_path, img_path, ...]
HybridNormalizedIoU.update() took 0.36s each time.
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis

array([0.12321951, 0.18997083, 0.20483381, 0.19901622, 0.1963652 ,
       0.19166286, 0.19165954, 0.19124712, 0.19124712, 0.19124712])

# Pixel ROC Precision Recall Metric

In [16]:
from sosmetrics.metrics import PixelROCPrecisionRecall
Metric = PixelROCPrecisionRecall(
    conf_thrs=10,
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
PixelROCPrecisionRecall.update() took 0.02s each time.
+--------------------+----------------------+-------------------------------------+
|      AUC_ROC       | AUC_PR(AUC function) | AP(BinaryAveragePrecision function) |
+--------------------+----------------------+-------------------------------------+
| 0.6083542704582214 |  0.5896512269973755  |              0.20967612             |
+--------------------+----------------------+-------------------------------------+
PixelROCPrecisionRecall
Test image of list [hwc, hwc, ...]
PixelROCPrecisionRecall.update() took 0.03s each time.
+--------------------+----------------------+-------------------------------------+
|      AUC_ROC       | AUC_PR(AUC function) | AP(BinaryAveragePrecision function) |
+--------------------+----------------------+-------------------------------------+
| 0.6083542704582214 |  0.5896512269973755

(0.6083542704582214,
 0.5896512269973755,
 array([1.5132346e-05, 1.5132346e-05, 1.5132346e-05, 1.5132346e-05,
        1.5132346e-05, 1.5132346e-05, 1.5132346e-05, 1.5132346e-05,
        1.5132346e-05, 1.0000000e+00], dtype=float32),
 array([0.21672702, 0.21672702, 0.21672702, 0.21672702, 0.21672702,
        0.21672702, 0.21672702, 0.21672702, 0.21672702, 1.        ],
       dtype=float32),
 array([0.00172775, 0.9612221 , 0.9612221 , 0.9612221 , 0.9612221 ,
        0.9612221 , 0.9612221 , 0.9612221 , 0.9612221 , 0.9612221 ,
        1.        ], dtype=float32),
 array([1.        , 0.21672702, 0.21672702, 0.21672702, 0.21672702,
        0.21672702, 0.21672702, 0.21672702, 0.21672702, 0.21672702,
        0.        ], dtype=float32),
 array(0.20967612, dtype=float32))

In [17]:
Metric.table

,AUC_ROC,AUC_PR,AP
0,0.608354,0.589651,0.209676


# Piexl Precision Recall F1 IoU 

In [18]:
from sosmetrics.metrics import PixelPrecisionRecallF1IoU
Metric = PixelPrecisionRecallF1IoU(
    conf_thr=0.5,
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
PixelPrecisionRecallF1IoU.update() took 0.02s each time.
+---------------+------------+--------+---------+
| Precision-0.5 | Recall-0.5 | F1-0.5 | IOU-0.5 |
+---------------+------------+--------+---------+
|     0.9612    |   0.2167   | 0.3537 | 0.21485 |
+---------------+------------+--------+---------+
PixelPrecisionRecallF1IoU
Test image of list [hwc, hwc, ...]
PixelPrecisionRecallF1IoU.update() took 0.03s each time.
+---------------+------------+--------+---------+
| Precision-0.5 | Recall-0.5 | F1-0.5 | IOU-0.5 |
+---------------+------------+--------+---------+
|     0.9612    |   0.2167   | 0.3537 | 0.21485 |
+---------------+------------+--------+---------+
PixelPrecisionRecallF1IoU
Test image_path of list, [img_path, img_path, ...]
PixelPrecisionRecallF1IoU.update() took 0.05s each time.
+---------------+------------+--------+---------+
| Precision-0.5 | Recall

(array([0.96122209]),
 array([0.21672702]),
 array([0.35370424]),
 array([0.21484854]))

In [19]:
Metric.table

,Precision-0.5,Recall-0.5,F1-0.5,IOU-0.5
0,0.961222,0.216727,0.353704,0.214849


In [20]:
from sosmetrics.metrics import TargetPdPixelFaROC
Metric = TargetPdPixelFaROC(
    dis_thrs=[1, 10],
    conf_thrs=10,
    match_alg='forloop',
    )

gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
TargetPdPixelFaROC.update() took 0.04s each time.
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr |  1.0   |  2.0   |  3.0   |  4.0   |  5.0   |  6.0   |  7.0   |  8.0   |  9.0   |  10.0  |
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| AUC-ROC | 0.2328 | 0.3711 | 0.4182 | 0.4340 | 0.4434 | 0.4452 | 0.4469 | 0.4454 | 0.4454 | 0.4405 |
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
TargetPdPixelFaROC(match_alg=forloop)
Test image of list [hwc, hwc, ...]
TargetPdPixelFaROC.update() took 0.05s each time.
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr |  1.0   |  2.0   |  3.0   |  4.0   |  5.0   |  6.0   |  7.0   |  8.0   |  9.0   |  10.0

(array([[0.        , 0.46540881, 0.46540881, 0.46540881, 0.46540881,
         0.46540881, 0.46540881, 0.46540881, 0.46540881, 0.46540881],
        [0.        , 0.74213836, 0.74213836, 0.74213836, 0.74213836,
         0.74213836, 0.74213836, 0.74213836, 0.74213836, 0.74213836],
        [0.        , 0.83647799, 0.83647799, 0.83647799, 0.83647799,
         0.83647799, 0.83647799, 0.83647799, 0.83647799, 0.83647799],
        [0.        , 0.86792453, 0.86792453, 0.86792453, 0.86792453,
         0.86792453, 0.86792453, 0.86792453, 0.86792453, 0.86792453],
        [0.        , 0.88679245, 0.88679245, 0.88679245, 0.88679245,
         0.88679245, 0.88679245, 0.88679245, 0.88679245, 0.88679245],
        [0.00628931, 0.89308176, 0.89308176, 0.89308176, 0.89308176,
         0.89308176, 0.89308176, 0.89308176, 0.89308176, 0.89308176],
        [0.01257862, 0.89937107, 0.89937107, 0.89937107, 0.89937107,
         0.89937107, 0.89937107, 0.89937107, 0.89937107, 0.89937107],
        [0.01886792, 0.8993